# **Finding Lane Lines on the Road** 


## We import the necessary libraries and pre-configuration

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.5/site-packages')

#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

## We define the image processing function we will use


In [ ]:
import math

#Transform an BGR image into a Grayscale images
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#Apply Canny edge algorithm on a selected image
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

#Apply Gaussian Blur algorithm on a selected image
def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

#Select a specific region in an image
def region_of_interest(img, vertices):
    
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=1):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    
    
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Apply a pipeline of functions on Images


In [ ]:
img = mpimg.imread('test_images/solidWhiteRight.jpg')
image_grayscale = grayscale(img)
image_canny = canny(image_grayscale,50,150)
image_blur = gaussian_blur(image_canny, 5)
vertices = np.array([[[0, 539],[900, 539],[475, 320]]])
image_region = region_of_interest(image_blur,vertices)
image_hough = hough_lines(image_region, 2, np.pi/180, 15, 40, 20)
image_w = weighted_img(image_hough, img, α=0.8, β=1., λ=0.)
plt.imshow(image_w, cmap='gray')

## Apply a pipeline of functions on Videos


In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.5/site-packages')

from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    image_grayscale = grayscale(image)
    image_canny = canny(image_grayscale,50,150)
    image_blur = gaussian_blur(image_canny, 5)
    vertices = np.array([[[0, 539],[900, 539],[475, 320]]])
    image_region = region_of_interest(image_blur,vertices)
    image_hough = hough_lines(image_region, 2, np.pi/180, 15, 40, 20)
    result = weighted_img(image_hough, image, α=0.8, β=1., λ=0.)
    return result

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Apply on the processing pipeline on the video 'Yellow.mp4'

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))